In [ ]:
import asyncio
import datetime
import ntplib
import socket
from tqdm import tqdm

from pydoll.browser import Chrome

# --- NTP 시간 동기화 함수 ---
async def get_time_offset(ntp_server='pool.ntp.org'):
    """NTP 서버와 로컬 시간의 오차를 초 단위로 반환합니다."""
    print(f"'{ntp_server}'에서 표준 시간 동기화를 시도합니다...")
    try:
        def request_ntp_time():
            ntp_client = ntplib.NTPClient()
            return ntp_client.request(ntp_server, version=3)
        response = await asyncio.to_thread(request_ntp_time)
        offset = response.offset
        print(f"성공: 로컬 시간과 표준 시간의 오차는 약 {offset:.4f}초 입니다.")
        return offset
    except (socket.gaierror, ntplib.NTPException) as e:
        print(f"경고: NTP 서버에 연결할 수 없습니다. 로컬 시간을 기준으로 동작합니다. 오류: {e}")
        return 0

# --- 메인 스크립트 ---
async def timed_refresh_example():
    print("Pydoll 브라우저를 시작하여 접속합니다...")
    async with Chrome() as browser:
        tab = await browser.start()
        await tab.go_to('https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07006&TeamCode=PF005')
        print("접속 완료. 목표 시간까지 대기합니다.")

        time_offset = await get_time_offset()
        offset_delta = datetime.timedelta(seconds=time_offset)

        base_target_time = datetime.datetime.now().replace(hour=14, minute=00, second=0, microsecond=0)
        
        # 2. 10 밀리초(ms) 먼저 실행하도록 시간 조정
        adjustment = datetime.timedelta(milliseconds=10)
        target_time = base_target_time - adjustment
        print(f"새로고침 목표 시간: {target_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        initial_seconds_left = (target_time - (datetime.datetime.now() + offset_delta)).total_seconds()
        
        if initial_seconds_left > 0:
            with tqdm(total=int(initial_seconds_left), desc="새로고침까지 대기 중...") as pbar:
                while True:
                    corrected_now = datetime.datetime.now() + offset_delta
                    
                    if corrected_now >= target_time:
                        pbar.n = pbar.total
                        pbar.refresh()
                        break

                    seconds_left = (target_time - corrected_now).total_seconds()
                    
                    pbar.n = pbar.total - int(seconds_left)
                    pbar.set_postfix_str(f"남은 시간: {seconds_left:.2f}초")
                    pbar.refresh()
                    
                    await asyncio.sleep(0.01)

        print("\n목표 시간이 되었습니다! 페이지를 새로고침합니다.")
        # --- 여기가 최종 수정된 부분 ---
        # pydoll Tab 객체의 내장 함수인 refresh()를 사용합니다.
        await tab.refresh()
        print("페이지를 새로고침했습니다.")

        print("작업 완료. 브라우저를 닫으려면 터미널에서 Enter 키를 누르세요...")
        await asyncio.to_thread(input)


# --- 코루틴 실행 ---
await timed_refresh_example()

Pydoll 브라우저를 시작하여 접속합니다...
접속 완료. 목표 시간까지 대기합니다.
'pool.ntp.org'에서 표준 시간 동기화를 시도합니다...
성공: 로컬 시간과 표준 시간의 오차는 약 -2.2960초 입니다.
새로고침 목표 시간: 2025-06-27 13:59:59


새로고침까지 대기 중...: 100%|██████████| 399/399 [06:39<00:00,  1.00s/it, 남은 시간: 0.00초]  



목표 시간이 되었습니다! 페이지를 새로고침합니다.
페이지를 새로고침했습니다.
작업 완료. 브라우저를 닫으려면 터미널에서 Enter 키를 누르세요...
